# Toric Decoder
Apply the [2D decoder](https://arxiv.org/abs/1406.2338), followed by [minimum-weight perfect matching](https://arxiv.org/abs/2303.15933), on the toric code with side length $L$. We consider i.i.d. $X$ errors on the qubits (bonds), and work with $m$ anyons (plaquettes). The errors are plotted on the dual lattice by default.
## 2D Decoder
At each time step out of $T=L$ total, perform the following steps:
1. Make an $X$ error on each qubit with probability $p=0.05$.
2. Perform $c$ field updates, according to the following rule:$$\phi\mapsto\phi+\frac\eta4\nabla^2\phi+q$$Where $\phi$ is the auxillary field, $\eta=0.1$ is the Jacobi smoothing parameter, and $q$ is the anyon density with unit mass $-\frac4\eta$. Take $4\pi G=1$, so that $\nabla^2\phi=\rho$.
3. For each anyon, move it its highest-$\phi$ neighboring cell with probability $\frac12$.

In [1]:
# %matplotlib widget

import numpy as np
import cupy as cp
mempool = cp.get_default_memory_pool()
# import os
# from tqdm.notebook import tqdm

from toric import State, pcm, mwpm, logical_error
from toric import decoder_2D
from pymatching import Matching

# import matplotlib.pyplot as plt
# plt.rcParams['figure.dpi'] = 150

from utils import get_gpu_memory

memory_baseline = get_gpu_memory() * (1024/1000)**2 # MB
print(memory_baseline)

def gpu():
    return get_gpu_memory() * (1024/1000)**2 - memory_baseline

128.97484799999998


In [2]:
# Simulation parameters

N = 5000 # Shots
L = 200 # Lattice size
p_error = 0.004 # Error probability per spin
η = 0.1 # Smoothing paramter for Jacobi method
c = 16 # "Field velocity" - number of field updates per cycle
T = 2 # Epochs, set to 2 for fast debug

matching = Matching(pcm(L))

In [3]:
state = State(N, L)

In [4]:
expected = (N/1000)*(L**2/1000)*7
reported = mempool.total_bytes() / 1000**2
actual = gpu()
print(f"Memory usage: {actual:.2f} MB (expected {expected:.2f} MB, reported {reported:.2f} MB)")

Memory usage: 1405.09 MB (expected 1400.00 MB, reported 1402.36 MB)


In [5]:
decoder_2D(state, T, c, η, p_error)

In [6]:
expected = (N/1000)*(L**2/1000)*16  
reported = mempool.total_bytes() / 1000**2
actual = gpu()
print(f"Memory usage: {actual:.2f} MB (expected {expected:.2f} MB, reported {reported:.2f} MB)")

Memory usage: 3208.64 MB (expected 3200.00 MB, reported 3202.36 MB)


In [7]:
x_correction, y_correction = mwpm(matching, state.q)
logical_error(x_correction ^ state.x_error, y_correction ^ state.y_error).mean()

array(0.)

In [8]:
expected = (N/1000)*(L**2/1000)*16
reported = mempool.total_bytes() / 1000**2
actual = gpu()
print(f"Memory usage: {actual:.2f} MB (expected {expected:.2f} MB, reported {reported:.2f} MB)")

Memory usage: 3208.64 MB (expected 3200.00 MB, reported 3202.36 MB)
